In [ ]:
__author__ = 'Benjamin Weaver <benjamin.weaver@noirlab.edu>'
__version__ = '20201123'
__datasets__ = ['sdss_dr16']
__keywords__ = ['HowTo','spectra','query','visualization','SDSS']

# Prospect + Specutils + Data Lab Spectrum Service

Project: Obtain SDSS spectra using the Data Lab spectrum service, convert data to [specutils](https://github.com/astropy/specutils) objects as needed, and use [prospect](https://github.com/desihub/prospect) to display the data.

Prospect is an *interactive* spectrum visualization service that is being actively used by the DESI project for visual inspection of commissioning and survey validation spectra right now.  Rather than just a simple *flux* versus *wavelength* plot, the interactive spectrum display provides pan, zoom, known spectral lines, targeting and other catalog information, metadata, *etc.*  The [Bokeh](https://bokeh.org) library handles the low-level interactivity.

We can leverage the generic spectrum container objects provided by [specutils](https://github.com/astropy/specutils) to allow Prospect to plot data from other surveys.  In this example, we are using [SDSS DR16](https://www.sdss.org/dr16/).

In the future we will extend this functionality to additional instruments.  For example, we are excited by the possibility of using Prospect and the Data Lab spectrum service for Gemini spectra.

*Caveat emptor:* Prospect is under active development, so it is not yet available as a standard Python package in the [PyPI repository](https://pypi.org).  An installable packages should be available as soon as December 2020.

In [ ]:
import os
import sys
# Manually install prospect for now.
sys.path.insert(0, os.path.join(os.environ['HOME'], 'Documents', 'Code', 'git', 'desihub', 'prospect', 'py'))

In [ ]:
import numpy as np
import astropy.units as u
from astropy.table import Table
from astropy.nddata import InverseVariance
from specutils import Spectrum1D, SpectrumCollection, SpectrumList
from prospect.specutils import read_spPlate, read_spZbest
from prospect.plotspecutils import plotspectra

In [ ]:
from dl import queryClient as qc, storeClient as sc
import specClient as spec

## SDSS/eBOSS spPlate file

SDSS spectra are stored per-plate in spPlate files.  These contain 640 spectra for the original SDSS spectrograph or 1000 spectra for the BOSS/eBOSS spectrograph.  All spPlate files have a common wavelength solution, so a spPlate file can be represented by a [Spectrum1D](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum1D.html#specutils.Spectrum1D) object.  This object is an input to the visualization system, so we need to get outputs from the spectrum service that match the inputs to the visualization.

## Find Plate 2955 in the Spectrum Service

### Plate Location

Here we find spectra corresponding to SDSS plate 2955, but using the spectrum service instead of reading the spectra from a file.

In [ ]:
plate = 2955
q = "SELECT ra, dec FROM sdss_dr16.platex WHERE plate = %d;" % plate
r = qc.query(sql=q, fmt='array')
r

### Spectrum Identifiers

SDSS plates are circular, so we'll do a cone search around the nominal plate center.  This gets us the spectrum identifiers we need to retrieve the spectra themselves.

In [ ]:
sdss_ids = spec.query(r[0], r[1], 1.5, fmt='array', out='', constraint='plate=2955 ORDER BY specobjid LIMIT 50')
sdss_ids

### Spectrum Service: Flux versus Wavelength

Now we fetch the spectra.  The returned data will be compacted into a single `Spectrum1D` object.

In [ ]:
%%time
sdss = spec.getSpec(sdss_ids, fmt='Spectrum1D', align=True)
sdss.mask = ~sdss.mask
sdss

### Redshift and other Metadata

We also need target and redshift information, which is stored in the SpecObj catalog.  Although all information is extractable from the database, we need to reformat it to match the expectations of Prospect.

In [ ]:
q = "SELECT specobjid, class, subclass, z, zerr, rchi2diff, zwarning FROM sdss_dr16.specobj WHERE specobjid IN ({0}) ORDER BY specobjid;".format(','.join(map(str, sdss_ids.tolist())))
sdss_z = qc.query(sql=q, fmt='table')
for c in ('specobjid', 'class', 'subclass', 'z', 'zerr', 'rchi2diff', 'zwarning'):
    if c == 'zerr':
        sdss_z.rename_column(c, 'Z_ERR')
    else:
        sdss_z.rename_column(c, c.upper())
sdss_z

In [ ]:
q = """SELECT s.specobjid, s.targetobjid, s.ra, s.dec,
    s.primtarget, s.sectarget,
    s.boss_target1,
    s.ancillary_target1, s.ancillary_target2,
    s.eboss_target0, s.eboss_target1, s.eboss_target2,
    p.u, p.g, p.r, p.i, p.z
FROM sdss_dr16.specobj AS s
LEFT JOIN sdss_dr16.photoplate AS p
ON s.bestobjid = p.objid
WHERE specobjid IN ({0})
ORDER BY specobjid;""".format(','.join(map(str, sdss_ids.tolist())))
sdss_plugmap = qc.query(sql=q, fmt='table')
for c in ('ra', 'dec', 'primtarget', 'sectarget',
          'boss_target1',
          'ancillary_target1', 'ancillary_target2',
          'eboss_target0', 'eboss_target1', 'eboss_target2'):
    sdss_plugmap.rename_column(c, c.upper())
sdss_plugmap['OBJID'] = np.vstack((np.bitwise_and(sdss_plugmap['targetobjid'] >> 32, 2**16 - 1).data,
                                   np.bitwise_and(sdss_plugmap['targetobjid'] >> 48, 2**11 - 1).data,
                                   np.bitwise_and(sdss_plugmap['targetobjid'] >> 29, 2**3 - 1).data,
                                   np.bitwise_and(sdss_plugmap['targetobjid'] >> 16, 2**12 - 1).data,
                                   np.bitwise_and(sdss_plugmap['targetobjid'], 2**16 - 1).data)).T
sdss_plugmap['MAG'] = np.vstack((np.where(np.isnan(sdss_plugmap['u'].data), 0, sdss_plugmap['u'].data),
                                 np.where(np.isnan(sdss_plugmap['g'].data), 0, sdss_plugmap['g'].data),
                                 np.where(np.isnan(sdss_plugmap['r'].data), 0, sdss_plugmap['r'].data),
                                 np.where(np.isnan(sdss_plugmap['i'].data), 0, sdss_plugmap['i'].data),
                                 np.where(np.isnan(sdss_plugmap['z'].data), 0, sdss_plugmap['z'].data))).T

sdss.meta['plugmap'] = sdss_plugmap
sdss_plugmap

## Launch the Visualization

In [ ]:
plotspectra(sdss.new_flux_unit(u.Unit('1e-17 erg/(cm**2 s Angstrom)')), zcatalog=sdss_z, model=(sdss.spectral_axis.value, sdss.meta['model']),
            notebook=True, title=os.path.basename('SDSS Plate %d' % plate),
            model_from_zcat=False, with_coaddcam=False, mask_type='PRIMTARGET', with_thumb_tab=False, with_vi_widgets=False)

## SDSS Spectra and Redshifts from Files

For reference, and to compare timing, below is how one would fetch spectra from a file.  In addition, we need the redshift catalog.

In [ ]:
os.environ['SPECTRO_REDUX'] = 'sdss_dr16://' + os.path.join('sdss', 'spectro', 'redux')
run2d = '26'
plate = '2955'
mjd = '54562'
sdss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
sdss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spZbest-{plate}-{mjd}.fits')
print(sdss_spectra)
print(sdss_redshifts)

In [ ]:
%%time
sdss_file = read_spPlate(sc.get(sdss_spectra, mode='fileobj'), limit=50)
sdss_file

In [ ]:
sdss_z, sdss_model = read_spZbest(sc.get(sdss_redshifts, mode='fileobj'), limit=50)

## eBOSS Spectra from Files

In [ ]:
run2d = 'v5_13_0'
plate = '9599'
mjd = '58131'
eboss_spectra = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, f'spPlate-{plate}-{mjd}.fits')
eboss_redshifts = os.path.join(os.environ['SPECTRO_REDUX'], run2d, plate, run2d, f'spZbest-{plate}-{mjd}.fits')
print(eboss_spectra)
print(eboss_redshifts)

In [ ]:
%%time
eboss = read_spPlate(sc.get(eboss_spectra, mode='fileobj'), limit=50)
eboss_z, eboss_model = read_spZbest(sc.get(eboss_redshifts, mode='fileobj'), limit=50)
eboss

In [ ]:
plotspectra(eboss, zcatalog=eboss_z, model=(eboss_model.spectral_axis.value, eboss_model.flux.value),
            notebook=True, title=os.path.basename(eboss_spectra),
            model_from_zcat=False, with_coaddcam=False, mask_type='EBOSS_TARGET1', with_thumb_tab=False, with_vi_widgets=False)

## Backup Code

In [ ]:
sdss = Spectrum1D(flux=np.vstack([s.flux for s in sdss_list]),
                  spectral_axis=sdss_list[0].spectral_axis,
                  uncertainty=InverseVariance(np.vstack([s.uncertainty.quantity for s in sdss_list])),
                  mask=np.vstack([s.mask for s in sdss_list]),
                  meta={'model': np.vstack([s.meta['model'] for s in sdss_list])*sdss_list[0].flux.unit})
sdss